# Manual Running End of Day (EOD)

This notebook demonstrates (using kdb q magic cells) how to connect to and manually execute the end of day (EOD) process on the RDB.

## Reference
[PyKX Jupyter q Magic Command](https://code.kx.com/pykx/2.5/getting-started/q_magic_command.html)

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from env import *
from managed_kx import *

# Cluster names and database
from basictick_setup import *

kx.q("\c 5000 5000")

# Specific cluster to connect to
CLUSTER_NAME = RDB_CLUSTER_NAME


In [2]:
# triggers credential get
session=None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using Defaults ...


# Execute End of Day (EOD) Function on RDB
Use components of the connection string and the pykx included q magic to connect to a cluster and interact with it, useful for remote debugging.

- Get the connection string   
  - must have permission to environment and cluster to get a connection string to the cluster
- Check state of the dataview _before_ running EOD
- Call the eod of day function on the RDB (.rdb.eod) with today's date
- Check state of the dataview _after_ EOD completes


In [3]:
# get the full connection string
conn_str = get_kx_connection_string(client, 
                                  environmentId=ENV_ID, clusterName=CLUSTER_NAME, 
                                   userName=KDB_USERNAME, boto_session=session)

# parse the connection string into its components
host, port, username, password = parse_connection_string(conn_str)

In [5]:
%%q --host $host --port $port --user $username --pass $password

/ current state of the dataview
dview:.aws.get_kx_dataview[.rdb.database;.rdb.dbView];
dview
""
/ dataview changeset details
.aws.get_changeset[.rdb.database;dview`changeset_id]

dataview_name          | "basictickdb_DBVIEW"
database_name          | "basictickdb"
status                 | "ACTIVE"
changeset_id           | "lsiXbka89ildRpVysIqEGg"
segment_configurations | +`db_paths`volume_name!(,,"/*";,"RDB_TP_SHARED")
availability_zone_id   | "use1-az6"
az_mode                | "SINGLE"
description            | "Dataview of database"
auto_update            | 0b
read_write             | 0b
active_versions        | +`changeset_id`segment_configurations`attached_clusters`created_timestamp`version_id!(,"lsiXbka89ildRpVysIqEGg";,+`db_paths`volume_name!(,,"/*";,"RDB_TP_SHARED");,,"HDB_basictickdb";,1.723068e+09;,"gsiXbnFvRwXXiSGiz8iPmA")
create_timestamp       | 1.723065e+09
last_modified_timestamp| 1.723069e+09

id    | "lsiXbka89ildRpVysIqEGg"
status| "COMPLETED"


In [6]:
%%q --host $host --port $port --user $username --pass $password

/ Run EOD for today
.u.end[.z.d]

In [7]:
%%q --host $host --port $port --user $username --pass $password

/ new state of dataview
dview:.aws.get_kx_dataview[.rdb.database;.rdb.dbView];
dview
""
/ dataview changeset details
.aws.get_changeset[.rdb.database;dview`changeset_id]

dataview_name          | "basictickdb_DBVIEW"
database_name          | "basictickdb"
status                 | "ACTIVE"
changeset_id           | "osiXfGYXMXzu6FdKu1Msag"
segment_configurations | +`db_paths`volume_name!(,,"/*";,"RDB_TP_SHARED")
availability_zone_id   | "use1-az6"
az_mode                | "SINGLE"
description            | "Dataview of database"
auto_update            | 0b
read_write             | 0b
active_versions        | +`changeset_id`segment_configurations`attached_clusters`created_timestamp`version_id!(,"osiXfGYXMXzu6FdKu1Msag";,+`db_paths`volume_name!(,,"/*";,"RDB_TP_SHARED");,,"HDB_basictickdb";,1.72307e+09;,"XsiXfOEp1zaf9T9CYAnU0w")
create_timestamp       | 1.723065e+09
last_modified_timestamp| 1.72307e+09

id    | "osiXfGYXMXzu6FdKu1Msag"
status| "COMPLETED"
